In [1]:
import torchvision.models as models
import os
from PIL import Image
import numpy as np
import collections
import time

import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader, TensorDataset, WeightedRandomSampler
from torchvision import transforms
import io
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print(os.listdir('/content/gdrive/MyDrive/RespiraCheck'))
print(os.path.exists('/content/gdrive/MyDrive/RespiraCheck/Cough Data/spectrograms/positive'))
print(os.path.exists('/content/gdrive/MyDrive/RespiraCheck/Cough Data/spectrograms/negative'))

Mounted at /content/gdrive
['Cough Data']
True
True


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models


class CNNModel(nn.Module):
    """A convolutional neural network model based on EfficientNet for spectrogram processing."""

    def __init__(self, dropout: float = 0.0):
        """Initializes the CNNModel using EfficientNet-B0 with an optional dropout layer.

        Args:
            dropout (float): Dropout probability before the final classification layer.
        """
        super(CNNModel, self).__init__()

        # Load EfficientNet-B0 with pre-trained weights
        self.efficientnet = models.efficientnet_b0(weights='IMAGENET1K_V1')

        # Get the number of features from the last layer of EfficientNet
        num_features = self.efficientnet.classifier[1].in_features

        # Replace the classifier with a new sequence including Dropout and FC layer
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=dropout),  # Dropout before classification layer
            nn.Linear(num_features, 1)  # Binary classification output
        )

        # Initialize the new FC layer weights
        nn.init.normal_(self.efficientnet.classifier[1].weight, mean=0.0, std=0.01)
        nn.init.zeros_(self.efficientnet.classifier[1].bias)

    def forward(self, spectrogram: torch.Tensor) -> torch.Tensor:
        """Defines the forward pass for EfficientNet with dropout.

        Args:
            spectrogram (torch.Tensor): Input tensor representing the spectrogram.

        Returns:
            torch.Tensor: The model's output (logit for binary classification).
        """
        return self.efficientnet(spectrogram)


In [4]:
class ModelHandler:
    """Handles the model training, evaluation, and inference pipeline.

    Attributes:
        device (torch.device): The device on which the model is executed (e.g., 'cpu' or 'cuda').
        model_path: Path to where .pth models should be saved.
    """

    def __init__(self,
                 model,
                 model_path: str,
                 optimizer: torch.optim.Optimizer,
                 loss_function: nn.Module,
                 steps_per_decay = 5,
                 lr_decay = 0.1):
        """Initializes the ModelHandler.

        Args:
            model_path (str | None): Path to the pre-trained model file (if available).
        """
        self.model = model
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model_path = model_path
        self.optimizer = optimizer
        self.lr_scheduler = opt.lr_scheduler.StepLR(self.optimizer, step_size=steps_per_decay, gamma=lr_decay)
        self.loss_function = loss_function

    def train_step(self, dataloader):
        """Trains the model for a single epoch.

        Args:
            dataloader (torch.utils.data.DataLoader): DataLoader for the training dataset.
        """
        self.model.train()
        avg_loss, acc = 0, 0
        for in_tensor, labels in dataloader:
            in_tensor, labels = in_tensor.to(self.device), labels.to(self.device)
            labels = labels.float().unsqueeze(1)  # Ensure correct shape for BCE loss

            logits = self.model(in_tensor) # Feed input into model

            loss = self.loss_function(logits, labels)  # Calculate loss
            avg_loss += loss.item()  # Add to cumulative loss

            # Gradient descent
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # Calculate batch accuracy and add it to cumulative accuracy
            prediction_classes = torch.round(torch.sigmoid(logits))
            batch_acc = torch.mean((prediction_classes == labels).float()).item()
            acc += batch_acc

        avg_loss /= len(dataloader)  # Calculate avg loss for epoch from cumulative loss
        acc /= len(dataloader)  # Calculate avg accuracy for epoch from cumulative accuracy
        train_results = {"avg_loss_per_batch": avg_loss, "avg_acc_per_batch": acc * 100}
        return train_results

    def val_step(self, dataloader):
        """Evaluates the model on the validation dataset.

        Args:
            dataloader (torch.utils.data.DataLoader): DataLoader for the validation dataset.
        """

        self.model.eval()
        with torch.inference_mode():
            avg_loss, acc = 0, 0
            for in_tensor, labels in dataloader:
                in_tensor, labels = in_tensor.to(self.device), labels.to(self.device)
                labels = labels.float().unsqueeze(1)  # Ensure correct shape for BCE loss

                logits = self.model(in_tensor)  # Feed input into model

                loss = self.loss_function(logits, labels)  # Calculate loss
                avg_loss += loss.item()  # Add to cumulative loss

                # Calculate batch accuracy and add it to cumulative accuracy
                prediction_classes = torch.round(torch.sigmoid(logits))
                batch_acc = torch.mean((prediction_classes == labels).float()).item()
                acc += batch_acc

            avg_loss /= len(dataloader)  # Calculate avg loss for each epoch from cumulative loss
            acc /= len(dataloader)  # Calculate avg accuracy for each epoch from cumulative accuracy
            valid_results = {"avg_loss_per_batch": avg_loss, "avg_acc_per_batch": acc * 100}
            return valid_results

    def train(self, train_loader, epochs: int, model_name: str):
        """Trains the model

        Args:
            train_loader: DataLoader for the training datasets
            epochs (int): Number of training epochs.
            model_name (str): Name to save the trained model.
        """
        self.model.to(self.device)
        training_results = {"epoch": [], "loss": [], "accuracy": []}
        validation_results = {"epoch": [], "loss": [], "accuracy": []}

        for epoch in range(epochs):

            # Train the model
            training_data = self.train_step(train_loader)
            training_results["epoch"].append(epoch)
            training_results["loss"].append(training_data["avg_loss_per_batch"])
            training_results["accuracy"].append(training_data["avg_acc_per_batch"])

            # Check the validation loss after training
            validation_data = self.val_step(val_loader)
            validation_results["epoch"].append(epoch)
            validation_results["loss"].append(validation_data["avg_loss_per_batch"])
            validation_results["accuracy"].append(validation_data["avg_acc_per_batch"])

            # Adjust learning rate if necessary
            if self.lr_scheduler:
                self.lr_scheduler.step()

            if epoch % 1 == 0:
                print(f"{epoch}:")
                print(f"LR: {self.optimizer.param_groups[0]['lr']}")
                print(f"Loss - {training_data['avg_loss_per_batch']:.5f} | Accuracy - {training_data['avg_acc_per_batch']:.2f}%")
                print(f"VLoss - {validation_data['avg_loss_per_batch']:.5f} | VAccuracy - {validation_data['avg_acc_per_batch']:.2f}%\n")

        self.save_model(model_state_dict=self.model.state_dict(), model_name=model_name)
        return training_results, validation_results


    def validate(self, val_loader, hyperparams: dict, save_best: bool = True) -> tuple[float, float]:
        """Validates the model on the validation dataset.

        Args:
            val_loader: DataLoader for the validation dataset.

        Returns:
            tuple: (validation accuracy, validation loss)
        """

        self.model.to(self.device)
        self.model.eval()

        val_losses_epoch, batch_sizes, accs = [], [], []
        best_acc = -1
        best_model_state = None  # Track the best model weights

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val = X_val.to(self.device)
                y_val = y_val.to(self.device).float().unsqueeze(1)

                y_prediction_val = self.model(X_val)  # forward pass
                loss = self.loss_function(y_prediction_val, y_val)
                val_losses_epoch.append(loss.item())

                # Compute accuracy
                y_prediction_val = torch.sigmoid(y_prediction_val)  # Convert logits to probabilities
                prediction_classes = (y_prediction_val > 0.5).float()  # Convert to binary 0/1

                acc = torch.mean((prediction_classes == y_val).float()).item()
                accs.append(acc)
                batch_sizes.append(X_val.shape[0])

        # Compute final validation loss and accuracy
        val_loss = np.mean(val_losses_epoch)
        val_acc = np.average(accs, weights=batch_sizes)  # Weighted average accuracy

        print(f'Validation accuracy: {val_acc*100:.2f}% | Validation loss: {val_loss:.4f}')

        if save_best and val_acc > best_acc:
            best_acc = val_acc
            best_model_state = self.model.state_dict()

            # Create model filename using hyperparameters
            hyperparam_str = "_".join(f"{key}:{value}" for key, value in hyperparams.items())
            model_filename = f"model_{hyperparam_str}_{time.time()}.pth"

            # Save the best model
            save_path = os.path.join(self.model_path, model_filename)
            torch.save(best_model_state, save_path)
            print(f"Best model saved at: {save_path}")
        return val_acc, val_loss


    def evaluate(self, test_loader) -> float:
        """Evaluates the model on the test dataset.

        Args:
            test_loader: DataLoader for the test dataset.
        """
        self.model.to(self.device)
        self.model.eval()
        batch_sizes, accs = [], []
        with torch.no_grad():
            for X_test, y_test, in test_loader:
                X_test = X_test.to(self.device)
                y_test = y_test.to(self.device)

                prediction = self.model(X_test)
                batch_sizes.append(X_test.shape[0])

                prediction = torch.sigmoid(prediction)
                prediction_classes = (prediction > 0.5).float() # This converts to binary classes 0 and 1

                acc = torch.mean((prediction_classes == y_test).float()).item()
                accs.append(acc)

        # Return average accuracy
        return 0.0 if not accs else np.average(accs, weights=batch_sizes)


    def predict(self, spectrogram: torch.Tensor, model_name: str) -> int:
        """Performs inference on a single spectrogram.

        Args:
            spectrogram (torch.Tensor): Input spectrogram for inference.

        Returns:
            torch.Tensor: The predicted output from the model.
        """
        self.load_model(self.model_path +f"/{model_name}")
        spectrogram = spectrogram.unsqueeze(0).to(self.device)

        with torch.no_grad:
            logits = self.model(spectrogram)

            probability = torch.sigmoid(logits)

            prediction = (probability > 0.5).float() # Turn probability into binary classificaiton

        return prediction.item()


    def save_model(self, model_state_dict: collections.OrderedDict, model_name: str | None) -> None:
        """Saves the model to the specified file path.

        Args:
            path (str): Path to save the model file.
        """
        path = self.model_path + "/" + model_name
        torch.save(model_state_dict, path)


    def load_model(self, path: str) -> None:
        """Loads a model from the specified file path.

        Args:
            path (str): Path to the model file.
        """
        self.model.load_state_dict(torch.load(path))
        self.model.to(self.device)
        self.model.eval()

In [5]:
class DataPipeline:
    """Processes datasets, including loading, splitting, and preparing for inference.

    This class provides methods for loading datasets, processing them for training,
    and preparing single instances for inference.

    Attributes:
        test_size (float): Proportion of the dataset to include in the test split.
        val_size (float): Proportion of the dataset to include for validation.
        audio_processor: AudioProcessor instance for handling audio processing.
        image_processor: ImageProcessor instance for handling spectrogram or extracted features processing.
    """

    def __init__(self, test_size: float, val_size: float):
        """Initializes the DatasetProcessor.

        Args:
            data_path (str): Path to the dataset file.
            test_size (float): Proportion of the dataset to include in the test split.
            audio_processor (AudioProcessor): Instance for handling audio processing.
            image_processor (ImageProcessor): Instance for handling spectrogram processing.
        """
        self.test_size = test_size
        self.val_size = val_size

    def load_dataset(self) -> TensorDataset:
        """Loads the dataset from the specified file path into a DataFrame."""
        tensors = []
        labels = []

        for label_folder, label_value in zip(["positive", "negative"], [1, 0]):
            spectrogram_folder = '/content/gdrive/MyDrive/RespiraCheck/Cough Data/spectrograms'
            output_dir = os.path.join(spectrogram_folder, label_folder)

            for image_name in tqdm(os.listdir(output_dir)):
                image_path = os.path.join(output_dir, image_name)
                image_tensor = self.image_to_tensor(image_path)

                tensors.append(image_tensor)
                labels.append(label_value)

        # Tensor of all features (N x D) - N is number of samples (377), D is feature dimension (3,224,224)
        X = torch.stack(tensors)
        # Tensor of all labels (N x 1) - 377x1
        y = torch.tensor(labels, dtype=torch.long)

        return TensorDataset(X, y)


    def image_to_tensor(self, image_path: str) -> torch.Tensor:
        """Converts a spectrogram image to a PyTorch tensor.

        Args:
            image_path (str): Path to the spectrogram image file.

        Returns:
            torch.Tensor: The PyTorch tensor representation of the image.
        """
        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to ResNet18 input size
            transforms.ToTensor(),  # Convert image to tensor
        ])

        image = Image.open(image_path).convert("RGB") # Convert from RGBA to RGB
        tensor_image = transform(image)

        return tensor_image  # shape will be 3, 224, 224

    def create_dataloaders(self, batch_size, dataset_path = None, upsample = True) -> tuple[DataLoader, DataLoader, DataLoader]:
        """Splits the dataset into training and test sets.

        Args:
            batch_size (int): The batch size for the DataLoader.
            dataset_path (str | None): Path to the TensorDataset file.

        Returns:
            tuple: (train_df, test_df) - The training and testing DataFrames.
        """
        if dataset_path:
            print(f"Loading dataset from {dataset_path}")
            dataset = torch.load(dataset_path, weights_only=False)
        else:
            print("Processing and loading dataset")
            dataset = self.load_dataset()

        # Calculate sizes
        test_size = round(self.test_size * len(dataset))
        val_size = round(self.val_size * len(dataset))
        train_size = round(len(dataset) - test_size - val_size)  # Remaining for training

        # Perform split
        train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

        # Upsample positive class
        if upsample:
            print("Upsampling data")
            labels = [label.item() for _, label in train_dataset]
            train_counts = {}
            for label in labels:
                train_counts[label] = train_counts.get(label, 0) + 1
            # print(train_counts)

            weights = torch.where(torch.tensor(labels) == 0, 1 / train_counts[0], 1 / train_counts[1])
            # print(labels[:5], weights[:5])

            wr_sampler = WeightedRandomSampler(weights, int(len(train_dataset) * 1.5))

            train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=wr_sampler)

        else:
            print("No upsampling")
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Create DataLoaders
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Count labels in train_loader
        train_counts = {}
        for _, labels in train_loader:
            for label in labels:
                train_counts[label.item()] = train_counts.get(label.item(), 0) + 1

        # print(train_counts)

        # Reduce memory footprint
        dataset, train_dataset, val_dataset, test_dataset = None, None, None, None

        return train_loader, val_loader, test_loader

In [6]:
import torch.optim as opt

# Static hyperparameters
EPOCHS = 20

# Learning rate scheduler
STEPS_PER_LR_DECAY = 20
LR_DECAY = 0.5

# Model parameters
DROPOUT = 0.5

# Training
LOSS_FN = nn.BCEWithLogitsLoss()



In [7]:
model = CNNModel(DROPOUT)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 104MB/s]


In [8]:
datapipeline = DataPipeline(test_size=0.15, val_size=0.15)
train_loader, val_loader, test_loader = datapipeline.create_dataloaders(batch_size=8)


Processing and loading dataset


100%|██████████| 4274/4274 [02:38<00:00, 26.94it/s]


Upsampling data


In [9]:
import torch.optim as opt
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchaudio.transforms as T
import torch.nn.utils
import numpy as np

# Define fixed hyperparameters
batch_size = 16  # Reduced batch size for better generalization
learning_rate = 0.0002  # More stable learning rate
weight_decay = 5e-4  # Regularization strength
dropout_rate = 0.6  # Increased dropout to reduce overfitting
patience = 15  # Increased patience for early stopping

print(f"\n🚀 Training with batch size: {batch_size}, learning rate: {learning_rate}")

# Initialize model (Use custom CNNModel with EfficientNet-B0 backbone)
cnn_model = CNNModel(dropout=dropout_rate)

# Choose optimizer (SGD with momentum for better generalization)
optimizer = opt.SGD(params=cnn_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

# Learning rate scheduler (CosineAnnealingLR for smooth LR decay)
scheduler = CosineAnnealingLR(optimizer, T_max=20)

# Data Augmentation (SpecAugment with additional Mixup)
def augment_spectrogram(spectrogram):
    spectrogram = T.FrequencyMasking(freq_mask_param=15)(spectrogram)
    spectrogram = T.TimeMasking(time_mask_param=25)(spectrogram)
    spectrogram = T.Vol(0.8)(spectrogram)  # Random volume adjustment
    return spectrogram

def mixup_data(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0))
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Create ModelHandler
model_handler = ModelHandler(model=cnn_model,
                             model_path="/content/gdrive/MyDrive/RespiraCheck/Cough Data",
                             optimizer=optimizer,
                             loss_function=LOSS_FN,
                             steps_per_decay=STEPS_PER_LR_DECAY,
                             lr_decay=LR_DECAY)

# Load dataset with the chosen batch size
datapipeline = DataPipeline(test_size=0.15, val_size=0.15)
train_loader, val_loader, test_loader = datapipeline.create_dataloaders(batch_size=batch_size, upsample=True)

# Debug: Check dataset distribution
print("\n📊 Dataset Split:")
print(f"- Training Samples: {len(train_loader.dataset)}")
print(f"- Validation Samples: {len(val_loader.dataset)}")
print(f"- Test Samples: {len(test_loader.dataset)}")

# Training loop with early stopping
best_val_loss = float("inf")
best_model = None
best_acc = 0.0
epochs_without_improvement = 0

for epoch in range(EPOCHS):
    print(f"\n🔄 Epoch {epoch+1}/{EPOCHS}")

    # Train
    train_results, val_results = model_handler.train(train_loader=train_loader, epochs=1, model_name="CNN_EfficientNet")
    train_loss = train_results["loss"][-1]  # Get the last recorded loss value

    # Validate
    val_acc, val_loss = model_handler.validate(val_loader, {"batch_size": batch_size, "lr": learning_rate})

    # Scheduler step
    scheduler.step()

    # Gradient Clipping to stabilize training
    torch.nn.utils.clip_grad_norm_(cnn_model.parameters(), max_norm=1.0)

    # Check if validation loss improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_acc = val_acc
        best_model = model_handler
        epochs_without_improvement = 0  # Reset counter if there's improvement
        print(f"✅ New best validation loss: {best_val_loss:.4f} | Validation accuracy: {val_acc*100:.2f}%")
    else:
        epochs_without_improvement += 1
        print(f"🔄 No improvement in validation loss for {epochs_without_improvement} epochs")

    # Early Stopping: Stop training if there's no improvement for `patience` epochs
    if epochs_without_improvement >= patience:
        print(f"⏹️ Early stopping triggered due to no improvement in validation loss.")
        break

# Final Testing
if best_model:
    test_acc = best_model.evaluate(test_loader)
    print(f"\n🎯 Test accuracy: {test_acc*100:.2f}% 🚀 Best model saved!")



🚀 Training with batch size: 16, learning rate: 0.0002
Processing and loading dataset


100%|██████████| 4274/4274 [01:46<00:00, 40.01it/s]


Upsampling data

📊 Dataset Split:
- Training Samples: 3979
- Validation Samples: 852
- Test Samples: 852

🔄 Epoch 1/20
0:
LR: 0.0002
Loss - 0.69308 | Accuracy - 51.96%
VLoss - 0.67317 | VAccuracy - 62.15%

Validation accuracy: 62.68% | Validation loss: 0.6732
Best model saved at: /content/gdrive/MyDrive/RespiraCheck/Cough Data/model_batch_size:16_lr:0.0002_1742504227.6740189.pth
✅ New best validation loss: 0.6732 | Validation accuracy: 62.68%

🔄 Epoch 2/20
0:
LR: 0.00019876883405951377
Loss - 0.67740 | Accuracy - 57.00%
VLoss - 0.67852 | VAccuracy - 56.25%

Validation accuracy: 56.34% | Validation loss: 0.6785
Best model saved at: /content/gdrive/MyDrive/RespiraCheck/Cough Data/model_batch_size:16_lr:0.0002_1742504259.0965977.pth
🔄 No improvement in validation loss for 1 epochs

🔄 Epoch 3/20
0:
LR: 0.00019510565162951537
Loss - 0.67556 | Accuracy - 58.16%
VLoss - 0.68158 | VAccuracy - 54.51%

Validation accuracy: 54.58% | Validation loss: 0.6816
Best model saved at: /content/gdrive/MyD

In [10]:
import torch
import torch.optim as opt
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchaudio.transforms as T
import torch.nn.utils
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

# Define fixed hyperparameters
batch_size = 16  # Reduced batch size for better generalization
learning_rate = 0.0002  # More stable learning rate
weight_decay = 5e-4  # Regularization strength
dropout_rate = 0.6  # Increased dropout to reduce overfitting
patience = 15  # Early stopping patience

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n🚀 Training with batch size: {batch_size}, learning rate: {learning_rate}")

# Initialize model (Use custom CNNModel with EfficientNet-B0 backbone)
cnn_model = CNNModel(dropout=dropout_rate).to(device)  # Move model to device

# Choose optimizer (SGD with momentum for better generalization)
optimizer = opt.SGD(params=cnn_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

# Learning rate scheduler (CosineAnnealingLR for smooth LR decay)
scheduler = CosineAnnealingLR(optimizer, T_max=20)

# Data Augmentation (SpecAugment + Mixup)
def augment_spectrogram(spectrogram):
    spectrogram = T.FrequencyMasking(freq_mask_param=15)(spectrogram)
    spectrogram = T.TimeMasking(time_mask_param=25)(spectrogram)
    spectrogram = T.Vol(0.8)(spectrogram)  # Random volume adjustment
    return spectrogram

def mixup_data(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0))
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Create ModelHandler
model_handler = ModelHandler(
    model=cnn_model,
    model_path="/content/gdrive/MyDrive/RespiraCheck/Cough Data",
    optimizer=optimizer,
    loss_function=LOSS_FN,
    steps_per_decay=STEPS_PER_LR_DECAY,
    lr_decay=LR_DECAY
)

# Load dataset with the chosen batch size
datapipeline = DataPipeline(test_size=0.15, val_size=0.15)
train_loader, val_loader, test_loader = datapipeline.create_dataloaders(batch_size=batch_size, upsample=True)

# Debug: Check dataset distribution
print("\n📊 Dataset Split:")
print(f"- Training Samples: {len(train_loader.dataset)}")
print(f"- Validation Samples: {len(val_loader.dataset)}")
print(f"- Test Samples: {len(test_loader.dataset)}")

# Function to Validate and Compute F1-score
def validate(model_handler, val_loader):
    model_handler.model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch in val_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)  # Move to device
            outputs = model_handler.model(inputs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    val_acc = accuracy_score(all_targets, all_preds)
    val_f1 = f1_score(all_targets, all_preds, average="weighted")

    return val_acc, val_f1

# Training loop with early stopping
best_val_loss = float("inf")
best_model = None
best_acc = 0.0
best_f1_score = 0.0
epochs_without_improvement = 0

for epoch in range(EPOCHS):
    print(f"\n🔄 Epoch {epoch+1}/{EPOCHS}")

    # Train
    train_results, val_results = model_handler.train(train_loader=train_loader, epochs=1, model_name="CNN_EfficientNet")

    # Validate and Compute F1-score
    val_acc, val_f1 = validate(model_handler, val_loader)

    # Scheduler step
    scheduler.step()

    print(f"📊 Validation Accuracy: {val_acc*100:.2f}% | F1-score: {val_f1:.4f}")

    # Check for improvement
    if val_f1 > best_f1_score:
        best_f1_score = val_f1
        best_model = model_handler
        epochs_without_improvement = 0  # Reset counter if there's improvement
        print(f"✅ New best F1-score: {best_f1_score:.4f}")
    else:
        epochs_without_improvement += 1
        print(f"🔄 No improvement in F1-score for {epochs_without_improvement} epochs")

    # Early Stopping
    if epochs_without_improvement >= patience:
        print(f"⏹️ Early stopping triggered due to no improvement in F1-score.")
        break

# Function to Evaluate Model on Test Set
def evaluate(model_handler, test_loader):
    model_handler.model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)  # Move to device
            outputs = model_handler.model(inputs)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_acc = accuracy_score(all_targets, all_preds)
    test_f1 = f1_score(all_targets, all_preds, average="weighted")

    print(f"\n🎯 Test Accuracy: {test_acc*100:.2f}% | Test F1-score: {test_f1:.4f} 🚀")
    return test_acc, test_f1

# Final Testing
if best_model:
    test_acc, test_f1 = evaluate(best_model, test_loader)
    print(f"\n🎯 Test Accuracy: {test_acc*100:.2f}% | Test F1-score: {test_f1:.4f} 🚀 Best model saved!")



🚀 Training with batch size: 16, learning rate: 0.0002
Processing and loading dataset


100%|██████████| 4274/4274 [01:46<00:00, 40.06it/s]


Upsampling data

📊 Dataset Split:
- Training Samples: 3979
- Validation Samples: 852
- Test Samples: 852

🔄 Epoch 1/20
0:
LR: 0.0002
Loss - 0.69127 | Accuracy - 51.96%
VLoss - 0.66768 | VAccuracy - 62.96%

📊 Validation Accuracy: 74.65% | F1-score: 0.6381
✅ New best F1-score: 0.6381

🔄 Epoch 2/20
0:
LR: 0.00019876883405951377
Loss - 0.67607 | Accuracy - 58.23%
VLoss - 0.64925 | VAccuracy - 66.20%

📊 Validation Accuracy: 74.65% | F1-score: 0.6381
🔄 No improvement in F1-score for 1 epochs

🔄 Epoch 3/20
0:
LR: 0.00019510565162951537
Loss - 0.66663 | Accuracy - 60.54%
VLoss - 0.62979 | VAccuracy - 68.98%

📊 Validation Accuracy: 74.65% | F1-score: 0.6381
🔄 No improvement in F1-score for 2 epochs

🔄 Epoch 4/20
0:
LR: 0.0001891006524188368
Loss - 0.65940 | Accuracy - 61.09%
VLoss - 0.63138 | VAccuracy - 66.32%

📊 Validation Accuracy: 74.65% | F1-score: 0.6381
🔄 No improvement in F1-score for 3 epochs

🔄 Epoch 5/20
0:
LR: 0.00018090169943749476
Loss - 0.65346 | Accuracy - 61.38%
VLoss - 0.61378